In [ ]:
import glob,os


# GET DIRECTORIES MISSING COPE DIRECTORIES
missing = []
completed = [] 
base = '/projects/niblab/bids_projects/Experiments/Bevel'
for b in sorted(glob.glob(os.path.join(base, "derivatives/sub-*"))):
    id_ = b.split("/")[-1]
    dirs_ = glob.glob(os.path.join(b, "func/Analysis/feat2/*.gfeat/cope*.feat"))
    if not dirs_:
        print("No cope directories found for .....", id_)
        missing.append(id_)
    else:
        completed.append(id_)

In [ ]:
import subprocess
bad_dict = {}
for id_ in missing:
    if id_ not in bad_dict:
        bad_dict[id_] = {
            "BAD_VOLS": [],
            "NIFTIS": [],
            "FMRIS": []
        }
    print("SUBJECT ANA: ", id_)
    niftis = glob.glob(os.path.join(base, "Nifti/%s/func/*run-*.nii.gz"%id_))
    fmris = glob.glob(os.path.join(base, "fmriprep/%s/func/*run-*_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"%id_))
    print("LENGTH RUNS: ", len(niftis))
    bad_dict[id_]["nifti_count"] = len(niftis)
    bad_dict[id_]["fmri_count"] = len(fmris)
    for f in fmris:
        vol_count = int(subprocess.check_output(['fslnvols', f]))
        if vol_count != 199:
            print(f.split("/")[-1], vol_count)
        
        
    for n in niftis:
        run_id = n.split("/")[-1].split("_")[2]
        bad_dict[id_]["NIFTIS"].append(n)
        vol_count = int(subprocess.check_output(['fslnvols', n]))
        if vol_count != 199:
            print("bad vols....", id_)
            print(run_id, vol_count)
            bad_dict[id_]["BAD_VOLS"].append(run_id)
        if ("5" or "6") in run_id:
            print("extra scans")
        elif len(niftis) < 4: 
            print("missing scans")
    
bad = []
            
for id_ in bad_dict:
    niftis = glob.glob(os.path.join(base, "Nifti/%s/func/*run-*.nii.gz"%id_))
    bad_vols = bad_dict[id_]["BAD_VOLS"]
    if not bad_vols:
        pass
    else:
        # ONLY ONE BAD VOLUME 
        if len(bad_vols) == 1:
            print("CASE1 for subject %s.....%s"%(id_, bad_vols))
            for a in bad_vols:
                bad_file = "%s_task-prob_%s_bold.nii.gz"%(id_, a)
                baddie = os.path.join(base, "Nifti/%s/func", bad_file)
            for n in niftis:
                if "run-4" in n:
                    orig = n
                    new = n.replace('run-4', 'run-3')
                    #os.rename(orig, new)
                elif "run-5" in n:
                    orig = n
                    new = n.replace("run-5", "run-4")
                    #os.rename(orig, new)
                #print("BAD ", baddie)
        else:
            print("BAD RUNS: ", bad_vols)
            print(len(bad_dict[id_]["NIFTIS"]))
            print("subject....",id_)
            bad.append(id_)
            #print("BAD FILES ++")
        
        
        